In [ ]:
!nvidia-smi

Mon Nov  1 22:04:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.9 MB 12.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.4 MB/s eta 0:00:35tcmalloc: large alloc 1147494400 bytes == 0x559a069d0000 @  0x7f2213e1e615 0x5599cc4654cc 0x5599cc54547a 0x5599cc4682ed 0x5599cc559e1d 0x5599cc4dbe99 0x5599cc4d69ee 0x5599cc469bda 0x5599cc4dbd00 0x5599cc4d69ee 0x5599cc469bda 0x5599cc4d8737 0x5599cc55ac66 0x5599cc4d7daf 0x5599cc55ac66 0x5599cc4d7daf 0x5599cc55ac66 0x5599cc4d7daf 0x5599cc46a039 0x5599cc4ad409 0x5599cc468c52 0x5599cc4dbc25 0x5599cc4d69ee 0x5599cc469bda 0x5599cc4d8737 0x5599cc4d69ee 0x5599cc469bda 0x5599cc4d7915 0x5599cc469afa 0x5599cc4d7c0d 0x5599cc4d69ee
     |████████████████████████████████| 881.9 MB 19 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 r

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
df_pat = pd.read_csv('/content/drive/MyDrive/Tesi/dataset_pat_bandPass2.csv')


In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 598041763 bytes (571 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip
--
Path = /content/drive/MyDrive/Tesi/dataset_pat_bandPass2.zip
Type = zip
Physical Size = 598041763

  0%      0% 4 - dataset_pat_bandPass2/132_2b2_Lr_mc_LittC2SE_bandPass2.wav                                                                     0% 9 - dataset_pat_bandPass2/147_1b3_Tc_mc_AKGC417L_bandPass2.wav

In [ ]:
print(f'Numero entry dataframe: {len(df_pat)} - Numero file dataset: 920')

Numero entry dataframe: 920 - Numero file dataset: 920


In [ ]:
df_pat

,filename,label
0,/content/dataset_pat_bandPass2/172_1b5_Tc_mc_A...,COPD
1,/content/dataset_pat_bandPass2/162_2b2_Al_mc_A...,COPD
2,/content/dataset_pat_bandPass2/139_1b1_Ar_sc_L...,COPD
3,/content/dataset_pat_bandPass2/176_1b3_Al_mc_A...,COPD
4,/content/dataset_pat_bandPass2/151_3p3_Ll_mc_A...,COPD
...,...,...
915,/content/dataset_pat_bandPass2/114_1b4_Lr_mc_A...,COPD
916,/content/dataset_pat_bandPass2/188_1b1_Ar_sc_M...,URTI
917,/content/dataset_pat_bandPass2/223_1b1_Pr_sc_M...,COPD
918,/content/dataset_pat_bandPass2/221_2b2_Al_mc_L...,COPD


In [ ]:
print(df_pat['label'].unique())

['COPD' 'Healthy' 'Pneumonia' 'Bronchiectasis' 'LRTI' 'URTI'
 'Bronchiolitis' 'Asthma']


------------------------------------
binary_balanced

In [ ]:
df_binary = df_pat.copy()
df_binary['label'] = (df_pat['label'] =='Healthy')*1
df_binary

,filename,label
0,/content/dataset_pat_bandPass2/172_1b5_Tc_mc_A...,0
1,/content/dataset_pat_bandPass2/162_2b2_Al_mc_A...,0
2,/content/dataset_pat_bandPass2/139_1b1_Ar_sc_L...,0
3,/content/dataset_pat_bandPass2/176_1b3_Al_mc_A...,0
4,/content/dataset_pat_bandPass2/151_3p3_Ll_mc_A...,0
...,...,...
915,/content/dataset_pat_bandPass2/114_1b4_Lr_mc_A...,0
916,/content/dataset_pat_bandPass2/188_1b1_Ar_sc_M...,0
917,/content/dataset_pat_bandPass2/223_1b1_Pr_sc_M...,0
918,/content/dataset_pat_bandPass2/221_2b2_Al_mc_L...,0


In [ ]:
df_binary.groupby('label').count()

,filename
label,
0,885
1,35


In [ ]:
#prova balanced
df = df_binary[df_binary.label==0]
rows = np.random.choice(df.index.values, 35)
sampled_df = df.loc[rows]
sampled_df.shape

(35, 2)

In [ ]:
df_binary = pd.concat([df_binary[df_binary.label==1], sampled_df],ignore_index=True)
df_binary.shape

(70, 2)

In [ ]:
df_binary

,filename,label
0,/content/dataset_pat_bandPass2/159_1b1_Ar_sc_M...,1
1,/content/dataset_pat_bandPass2/136_1b1_Ar_sc_M...,1
2,/content/dataset_pat_bandPass2/159_1b1_Al_sc_M...,1
3,/content/dataset_pat_bandPass2/159_1b1_Ll_sc_M...,1
4,/content/dataset_pat_bandPass2/127_1b1_Ar_sc_M...,1
...,...,...
65,/content/dataset_pat_bandPass2/170_1b4_Tc_mc_A...,0
66,/content/dataset_pat_bandPass2/120_1b1_Al_sc_M...,0
67,/content/dataset_pat_bandPass2/198_1b5_Pr_mc_A...,0
68,/content/dataset_pat_bandPass2/130_1p3_Tc_mc_A...,0


In [ ]:
df_binary.groupby('label').count()

,filename
label,
0,35
1,35


In [ ]:
save_path = '/content/drive/MyDrive/Tesi'
train_binary, test_binary = train_test_split(df_binary, test_size=0.2, random_state=42)#, stratify=df["label"])

train_binary = train_binary.reset_index(drop=True)
test_binary = test_binary.reset_index(drop=True)

train_binary.to_csv(f"{save_path}/train_binary_pat_balanced.csv",  index=False)
test_binary.to_csv(f"{save_path}/test_binary_pat_balanced.csv",  index=False)


print(train_binary.shape)
print(test_binary.shape)

(56, 2)
(14, 2)


----------------------------------
3-class balanced


In [ ]:
df_multiclass = df_pat.copy()
df_multiclass.loc[df_multiclass.label == "Healthy", "label"] = 0

df_multiclass.loc[df_multiclass.label == "COPD", "label"] = 1
df_multiclass.loc[df_multiclass.label == "Bronchiectasis", "label"] = 1
df_multiclass.loc[df_multiclass.label == "Asthma", "label"] = 1

df_multiclass.loc[df_multiclass.label == "LRTI", "label"] = 2
df_multiclass.loc[df_multiclass.label == "URTI", "label"] = 2
df_multiclass.loc[df_multiclass.label == "Pneumonia", "label"] = 2
df_multiclass.loc[df_multiclass.label == "Bronchiolitis", "label"] = 2
df_multiclass

,filename,label
0,/content/dataset_pat_bandPass2/172_1b5_Tc_mc_A...,1
1,/content/dataset_pat_bandPass2/162_2b2_Al_mc_A...,1
2,/content/dataset_pat_bandPass2/139_1b1_Ar_sc_L...,1
3,/content/dataset_pat_bandPass2/176_1b3_Al_mc_A...,1
4,/content/dataset_pat_bandPass2/151_3p3_Ll_mc_A...,1
...,...,...
915,/content/dataset_pat_bandPass2/114_1b4_Lr_mc_A...,1
916,/content/dataset_pat_bandPass2/188_1b1_Ar_sc_M...,2
917,/content/dataset_pat_bandPass2/223_1b1_Pr_sc_M...,1
918,/content/dataset_pat_bandPass2/221_2b2_Al_mc_L...,1


In [ ]:
df_multiclass.groupby('label').count()

,filename
label,
0,35
1,810
2,75


In [ ]:
df_pat.groupby('label').count()

,filename
label,
Asthma,1
Bronchiectasis,16
Bronchiolitis,13
COPD,793
Healthy,35
LRTI,2
Pneumonia,37
URTI,23


In [ ]:
#prova balanced
df = df_multiclass[df_multiclass.label==1]
rows = np.random.choice(df.index.values, 50)
sampled_df = df.loc[rows]
sampled_df.shape

(50, 2)

In [ ]:
df_multiclass = pd.concat([df_multiclass[df_multiclass.label==0], sampled_df,df_multiclass[df_multiclass.label==2]], ignore_index=True)
df_multiclass.shape

(160, 2)

In [ ]:
df_multiclass

,filename,label
0,/content/dataset_pat_bandPass2/159_1b1_Ar_sc_M...,0
1,/content/dataset_pat_bandPass2/136_1b1_Ar_sc_M...,0
2,/content/dataset_pat_bandPass2/159_1b1_Al_sc_M...,0
3,/content/dataset_pat_bandPass2/159_1b1_Ll_sc_M...,0
4,/content/dataset_pat_bandPass2/127_1b1_Ar_sc_M...,0
...,...,...
155,/content/dataset_pat_bandPass2/105_1b1_Tc_sc_M...,2
156,/content/dataset_pat_bandPass2/140_2b3_Tc_mc_L...,2
157,/content/dataset_pat_bandPass2/165_1b1_Pr_sc_M...,2
158,/content/dataset_pat_bandPass2/161_1b1_Pl_sc_M...,2


In [ ]:
df_multiclass.groupby('label').count()

,filename
label,
0,35
1,50
2,75


In [ ]:
save_path = '/content/drive/MyDrive/Tesi'
train_multiclass, test_multiclass = train_test_split(df_multiclass, test_size=0.2, random_state=42)#, stratify=df["label"])

train_multiclass = train_multiclass.reset_index(drop=True)
test_multiclass = test_multiclass.reset_index(drop=True)

train_multiclass.to_csv(f"{save_path}/train_multiclass_pat_balanced.csv",  index=False)
test_multiclass.to_csv(f"{save_path}/test_multiclass_pat_balanced.csv",  index=False)


print(train_multiclass.shape)
print(test_multiclass.shape)

(128, 2)
(32, 2)


------------------------------

In [ ]:
train_binary

In [ ]:
import librosa
total_duration=0
num_file=0
for elem,label in tqdm(zip(df_binary['filename'], df_binary['label']), total=df_binary.shape[0]):
  total_duration += librosa.get_duration(filename=elem)
  num_file = num_file+1

print(total_duration/num_file)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/70 [00:00<?, ?it/s]

21.381801101392938


In [ ]:
total_duration/60

24.94543461829176

In [ ]:
!pip -q install torchaudio
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 33.6 MB/s 
     |████████████████████████████████| 596 kB 47.5 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torchaudio
duration = 20
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result
        
        return result

In [ ]:
ds = AudiosDataset(df_binary['filename'], df_binary['label'])
for i in ds:
  print(i)
  print(i['representation'].shape)
  print(type(i['representation']))
  break

{'target': 1, 'representation': tensor([[-2.8818e-05, -2.9719e-05, -7.0477e-05,  ..., -2.8184e-04,
          8.5316e-05,  5.8350e-04]])}
torch.Size([1, 320000])
<class 'torch.Tensor'>


In [ ]:
save_path = '/content/drive/MyDrive/Tesi'
train_binary, test_binary = train_test_split(df_binary, test_size=0.2, random_state=42)#, stratify=df["label"])

train_binary = train_binary.reset_index(drop=True)
test_binary = test_binary.reset_index(drop=True)

train_binary.to_csv(f"{save_path}/train_binary.csv", sep="\t", index=False)
test_binary.to_csv(f"{save_path}/test_binary.csv", sep="\t", index=False)


print(train_binary.shape)
print(test_binary.shape)

(56, 2)
(14, 2)


In [ ]:
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install fairseq@git+https://github.com//pytorch/fairseq.git@f2146bdc7abf293186de9449bfa2272775e39e1d#egg=fairseq
!pip install sox

  Cloning https://github.com//pytorch/fairseq.git (to revision f2146bdc7abf293186de9449bfa2272775e39e1d) to /tmp/pip-install-sjo0yr_o/fairseq_49421965ef764380b31d5a83744640f6
  Running command git clone -q https://github.com//pytorch/fairseq.git /tmp/pip-install-sjo0yr_o/fairseq_49421965ef764380b31d5a83744640f6
  Running command git rev-parse -q --verify 'sha^f2146bdc7abf293186de9449bfa2272775e39e1d'
  Running command git fetch -q https://github.com//pytorch/fairseq.git f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git checkout -q f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 6.7 M

In [ ]:
!pip install transformers

In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for aalbert classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 2)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features[0]
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime


def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/Hubert/PROVA-BALANCED_bandPass2_dropout=0.5_PATHOLOGY-healthy-unhealthy_hubert_lr={lr}_35-35_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

---------------------------

------------------------

#### Training Hubert pathology driven binary

In [ ]:
freq=16
duration=20
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 2
num_workers = 2
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
from transformers import AutoModel
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.83 #nan->0: 100%|██████████| 28/28 [00:39<00:00,  1.42s/it]


loss: 0.681824
acc: 0.553571
start validation


Loss: 0.62 #nan->0: 100%|██████████| 7/7 [00:03<00:00,  1.78it/s]


validation loss: 0.6846623420715332
validation accuracy: 0.35714285714285715
Save new model!- 22/10 h23:15
--------------------
* Epoch 2/100


...

validation loss: 0.04261569306254387
validation accuracy: 1.0
--------------------
* Epoch 29/100


Loss: 0.15 #nan->0: 100%|██████████| 28/28 [00:40<00:00,  1.43s/it]


loss: 0.280335
acc: 0.892857
start validation


Loss: 0.02 #nan->0: 100%|██████████| 7/7 [00:03<00:00,  1.76it/s]

validation loss: 0.0530649796128273
validation accuracy: 1.0
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.69 #nan->0: 100%|██████████| 30/30 [00:42<00:00,  1.40s/it]


loss: 0.682832
acc: 0.500000
start validation


Loss: 0.73 #nan->0: 100%|██████████| 7/7 [00:03<00:00,  1.83it/s]


validation loss: 0.6744509339332581
validation accuracy: 0.6428571428571429
Save new model!- 21/10 h15:10
--------------------
* Epoch 2/100


...

validation loss: 0.31629374623298645
validation accuracy: 0.8571428571428571
--------------------
* Epoch 27/100


Loss: 0.03 #nan->0: 100%|██████████| 30/30 [00:42<00:00,  1.42s/it]


loss: 0.103738
acc: 0.966667
start validation


Loss: 0.65 #nan->0: 100%|██████████| 7/7 [00:03<00:00,  1.85it/s]

validation loss: 0.42310959100723267
validation accuracy: 0.7857142857142857
--------------------


### Tentativi precedenti

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/PROVA-BALANCED_bandPass2_dropout=0.5_PATHOLOGY-healthy-unhealthy_hubert_lr=1e-05_35-35_batch=2_16mhz_20sec.bin'))

<All keys matched successfully>

In [ ]:
def classify(model, dataset, path_model, task='Non hai scritto il tipo di classificazione'):
  #task è una stringa che dice il tipo di classificazione 
  correct = 0
  total = 0
  predictions = [] 
  y_true = [] 
  with torch.no_grad():
    progress = tqdm(dataset)
    for batch in progress:
      audio, labels = batch["representation"], batch["target"]
      audio = audio.to(DEVICE)
      labels = labels.to(DEVICE)
      outputs = model(audio)
      _, predicted = torch.max(outputs, dim=1)
      total += labels.size(0)
      y_true.extend(labels.cpu().numpy())
      predictions.extend(predicted.cpu().numpy())
      correct += (predicted == labels).sum().item()

  print(f"\n Classification report del modello {path_model.split('/')[5]} per la classificazione {task}")
  
  return y_true, predictions

In [ ]:
path_model = "/content/drive/MyDrive/Tesi/Hubert/PROVA-BALANCED_bandPass2_dropout=0.5_PATHOLOGY-healthy-unhealthy_hubert_lr=1e-05_35-35_batch=2_16mhz_20sec.bin"
y_true, y_pred = classify(model, loaders["test"], path_model, '4-CLASS CLASSIFICATION')

report = classification_report(y_true, y_pred)
print(report)

100%|██████████| 7/7 [00:03<00:00,  1.91it/s]


 Classification report del modello Hubert per la classificazione 4-CLASS CLASSIFICATION


NameError: ignored

--------------------------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =5e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.08 #nan->0: 100%|██████████| 368/368 [08:35<00:00,  1.40s/it]


loss: 0.256119
acc: 0.955163
start validation


Loss: 0.04 #nan->0: 100%|██████████| 92/92 [00:46<00:00,  1.99it/s]


validation loss: 0.1969015896320343
validation accuracy: 0.9510869565217391
Save new model!- 20/10 h3:24
--------------------
* Epoch 2/100


...

Loss: 0.01 #nan->0: 100%|██████████| 368/368 [08:38<00:00,  1.41s/it]


loss: 0.121117
acc: 0.964674
start validation


Loss: 0.12 #nan->0: 100%|██████████| 92/92 [00:46<00:00,  1.98it/s]


validation loss: 0.17127646505832672
validation accuracy: 0.9510869565217391
--------------------
* Epoch 8/100


Loss: 0.16 #nan->0:  22%|██▏       | 80/368 [01:54<06:52,  1.43s/it]

In [ ]:
def classify(model, dataset):
  correct = 0
  total = 0
  predictions = [] 
  y_true = [] 
  with torch.no_grad():
      progress = tqdm(dataset)
      for batch in progress:
        # sys.stdout.write(
        #   f"\r{batch_i}/{len(dataset)}"
        #   )
        audio, labels = batch["representation"], batch["target"]
        audio = audio.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = model(audio)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.size(0)
        y_true.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())
        correct += (predicted == labels).sum().item()

  print(f'\n Accuracy della rete composta da {len(dataset)*batch_size} audio: {100* correct/total}')
  
  return y_true, predictions


In [ ]:
from sklearn.metrics import classification_report
y_true, y_pred = classify(model, loaders["train"])

# print(classification_report(y_true, y_pred, target_names=target_names))
report = classification_report(y_true, y_pred)
print(report)

100%|██████████| 92/92 [00:49<00:00,  1.85it/s]


 Accuracy della rete composta da 736 audio: 96.46739130434783
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       710
           1       0.00      0.00      0.00        26

    accuracy                           0.96       736
   macro avg       0.48      0.50      0.49       736
weighted avg       0.93      0.96      0.95       736




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Training Wav2vec2 augmented

In [ ]:
df_rc = pd.read_csv('/content/drive/MyDrive/Tesi/dataset_binary_augmented.csv')

In [ ]:
#extract with 7zip and progressbar to VM Colab
!7z x /content/drive/MyDrive/Tesi/augmented_sample.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 3408302740 bytes (3251 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/augmented_sample.zip
 68% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/augmented_sample.zip
Type = zip
Physical Size = 3408302740

  0%      0% 15 - augmented_sample/198_6p1_Pl_mc_AKGC417L_3_augmented.wav                                                                   0% 22 - augmented_sample/193_1b4_Lr_mc_AKGC417L_7_augmented.wav

In [ ]:
print(f'Numero entry dataframe: {len(df_rc)} - Numero file dataset: 12898')

Numero entry dataframe: 12898 - Numero file dataset: 12898


In [ ]:
df_rc.groupby('label').count()

,filename
label,
0,6256
1,6642


In [ ]:
save_path = '/content/drive/MyDrive/Tesi'
train_binary_augmented, test_binary_augmented = train_test_split(df_rc, test_size=0.2, random_state=42)#, stratify=df["label"])

train_binary_augmented = train_binary_augmented.reset_index(drop=True)
test_binary_augmented = test_binary_augmented.reset_index(drop=True)

train_binary_augmented.to_csv(f"{save_path}/train_binary_augmented.csv", sep="\t", index=False)
test_binary_augmented.to_csv(f"{save_path}/test_binary_augmented.csv", sep="\t", index=False)


print(train_binary_augmented.shape)
print(test_binary_augmented.shape)

(10318, 2)
(2580, 2)


-----------------------

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=5
train_ds = AudiosDataset(train_binary_augmented['filename'], train_binary_augmented['label'])
test_ds = AudiosDataset(test_binary_augmented['filename'], test_binary_augmented['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h") #facebook/wav2vec2-base-960h o facebook/wav2vec2-base-100k-voxpopuli o wav2vec2-large-xlsr-53-greek o m3hrdadfi/wav2vec2-base-100k-eating-sound-collection

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 wav2vec2

* Epoch 1/100


Loss: 0.67 #nan->0: 100%|██████████| 1289/1289 [26:00<00:00,  1.21s/it]


loss: 0.694367
acc: 0.521237
start validation


Loss: 0.65 #nan->0: 100%|██████████| 322/322 [02:22<00:00,  2.27it/s]


validation loss: 0.6716501712799072
validation accuracy: 0.5877329192546584
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.58 #nan->0: 100%|██████████| 1289/1289 [25:59<00:00,  1.21s/it]


loss: 0.595571
acc: 0.687645
start validation


Loss: 0.40 #nan->0: 100%|██████████| 322/322 [02:22<00:00,  2.26it/s]


validation loss: 0.6048697829246521
validation accuracy: 0.6832298136645962
Save new model!
--------------------
* Epoch 8/100


Loss: 0.63 #nan->0:  64%|██████▍   | 830/1289 [16:48<09:21,  1.22s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/OK-healthy-unhealthy_wav2vec2_lr=1e-05_batch=8_16mhz_5sec_augmented.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True, last_epoch=7)
#16mhz 5secondi tanh dropout=0.7 batch=8 wav2vec2

* Epoch 8/100


  0%|          | 0/1289 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.41 #nan->0: 100%|██████████| 1289/1289 [25:51<00:00,  1.20s/it]


loss: 0.576366
acc: 0.703258
start validation


Loss: 0.57 #nan->0: 100%|██████████| 322/322 [02:20<00:00,  2.29it/s]


validation loss: 0.613784909248352
validation accuracy: 0.6972049689440993
Save new model!
--------------------
* Epoch 9/100


...

Loss: 0.34 #nan->0: 100%|██████████| 1289/1289 [25:51<00:00,  1.20s/it]


loss: 0.505577
acc: 0.755334
start validation


Loss: 0.52 #nan->0: 100%|██████████| 322/322 [02:20<00:00,  2.29it/s]


validation loss: 0.6118766069412231
validation accuracy: 0.7146739130434783
--------------------
* Epoch 13/100


Loss: 0.31 #nan->0:  15%|█▍        | 189/1289 [03:51<23:01,  1.26s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/OK-healthy-unhealthy_wav2vec2_lr=1e-05_batch=8_16mhz_5sec_augmented.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True, last_epoch=11)
#16mhz 5secondi tanh dropout=0.7 batch=8 wav2vec2

* Epoch 12/100


  0%|          | 0/1289 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.19 #nan->0: 100%|██████████| 1289/1289 [27:13<00:00,  1.27s/it]


loss: 0.488460
acc: 0.768037
start validation


Loss: 0.41 #nan->0: 100%|██████████| 322/322 [02:27<00:00,  2.18it/s]


validation loss: 0.5954684019088745
validation accuracy: 0.7142857142857143
Save new model!
--------------------
* Epoch 13/100


...

validation loss: 0.6157023310661316
validation accuracy: 0.7286490683229814
--------------------
* Epoch 17/100


Loss: 0.47 #nan->0: 100%|██████████| 1289/1289 [27:12<00:00,  1.27s/it]


loss: 0.391790
acc: 0.824088
start validation


Loss: 0.38 #nan->0: 100%|██████████| 322/322 [02:27<00:00,  2.18it/s]

validation loss: 0.6294905543327332
validation accuracy: 0.7263198757763976
--------------------


----------------------

In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


  0%|          | 0/1289 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.69 #nan->0: 100%|██████████| 1289/1289 [27:17<00:00,  1.27s/it]


loss: 0.703050
acc: 0.496606
start validation


Loss: 0.69 #nan->0: 100%|██████████| 322/322 [02:27<00:00,  2.18it/s]


validation loss: 0.6928600072860718
validation accuracy: 0.5190217391304348
Save new model!
--------------------
* Epoch 2/100


Loss: 0.74 #nan->0:  59%|█████▊    | 757/1289 [15:59<11:14,  1.27s/it]

-----------------------------

In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =5e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

#### Training Hubert not augmented

**bandPass**

In [ ]:
freq=16
duration=3
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 2
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

-------------------------
**rete deep e dropout a 0.3**

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.58 #nan->0: 100%|██████████| 689/689 [08:17<00:00,  1.39it/s]


loss: 0.660326
acc: 0.598331
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:59<00:00,  2.90it/s]


validation loss: 0.6229278445243835
validation accuracy: 0.6475290697674418
Save new model!- 18/10 h21:51
--------------------
* Epoch 2/100


...

Loss: 0.18 #nan->0: 100%|██████████| 689/689 [08:16<00:00,  1.39it/s]


loss: 0.476834
acc: 0.774855
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:59<00:00,  2.89it/s]


validation loss: 0.49236005544662476
validation accuracy: 0.7623546511627907
Save new model!- 18/10 h22:47
--------------------
* Epoch 8/100


Loss: 0.42 #nan->0:  13%|█▎        | 92/689 [01:06<07:09,  1.39it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass2_deep_dropout=0.3_healthy-unhealthy_hubert_base_lr=1e-05_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=7)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 8/100


Loss: 0.32 #nan->0: 100%|██████████| 689/689 [08:25<00:00,  1.36it/s]


loss: 0.447739
acc: 0.795718
start validation


Loss: 0.86 #nan->0: 100%|██████████| 172/172 [01:21<00:00,  2.12it/s]


validation loss: 0.5218931436538696
validation accuracy: 0.752906976744186
Save new model!- 18/10 h23:24
--------------------
* Epoch 9/100


...

validation loss: 0.5836959481239319
validation accuracy: 0.7739825581395349
--------------------
* Epoch 17/100


Loss: 0.27 #nan->0: 100%|██████████| 689/689 [08:26<00:00,  1.36it/s]


loss: 0.301967
acc: 0.869557
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [01:22<00:00,  2.08it/s]

validation loss: 0.544214129447937
validation accuracy: 0.7877906976744186
--------------------


-------------------------------
**bandPass2 dataset - tokenizer**

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.57 #nan->0: 100%|██████████| 689/689 [08:14<00:00,  1.39it/s]


loss: 0.683333
acc: 0.563135
start validation


Loss: 0.56 #nan->0: 100%|██████████| 172/172 [01:02<00:00,  2.75it/s]


validation loss: 0.6365666389465332
validation accuracy: 0.6322674418604651
Save new model!- 18/10 h19:6
--------------------
* Epoch 2/100


...

validation loss: 0.6336488723754883
validation accuracy: 0.747093023255814
--------------------
* Epoch 16/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [08:15<00:00,  1.39it/s]


loss: 0.360975
acc: 0.851778
start validation


Loss: 0.16 #nan->0: 100%|██████████| 172/172 [01:03<00:00,  2.72it/s]


validation loss: 0.5662387013435364
validation accuracy: 0.778343023255814
--------------------


--------------------

**dropout=0.5**

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.69 #nan->0: 100%|██████████| 689/689 [08:42<00:00,  1.32it/s]


loss: 0.670677
acc: 0.584724
start validation


Loss: 0.55 #nan->0: 100%|██████████| 172/172 [01:20<00:00,  2.15it/s]


validation loss: 0.6388217806816101
validation accuracy: 0.654796511627907
Save new model!- 17/10 h18:46
--------------------
* Epoch 2/100


...

Loss: 0.82 #nan->0: 100%|██████████| 689/689 [08:48<00:00,  1.30it/s]


loss: 0.509508
acc: 0.751996
start validation


Loss: 0.45 #nan->0: 100%|██████████| 172/172 [01:20<00:00,  2.13it/s]


validation loss: 0.5139895081520081
validation accuracy: 0.7638081395348837
--------------------
* Epoch 7/100


Loss: 0.48 #nan->0:  82%|████████▏ | 567/689 [07:13<01:33,  1.31it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_dropout=0.5_healthy-unhealthy_hubert_base_lr=1e-05_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=4)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 5/100


Loss: 0.79 #nan->0: 100%|██████████| 689/689 [08:47<00:00,  1.31it/s]


loss: 0.535535
acc: 0.737482
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [01:18<00:00,  2.20it/s]


validation loss: 0.5542703866958618
validation accuracy: 0.720203488372093
Save new model!- 17/10 h23:36
--------------------
* Epoch 6/100


...

validation loss: 0.550632655620575
validation accuracy: 0.7885174418604651
--------------------
* Epoch 16/100


Loss: 0.07 #nan->0: 100%|██████████| 689/689 [08:43<00:00,  1.32it/s]


loss: 0.316038
acc: 0.867925
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [01:20<00:00,  2.14it/s]


validation loss: 0.5494030117988586
validation accuracy: 0.7906976744186046
--------------------


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_dropout=0.5_87%-79%_healthy-unhealthy_hubert_base_lr=1e-05_batch=8_16mhz_3sec.bin')


In [ ]:
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=16)
#DA QUI. FAI QUALCHE ALTRA EPOCA

* Epoch 17/100


Loss: 0.22 #nan->0: 100%|██████████| 689/689 [08:42<00:00,  1.32it/s]


loss: 0.301435
acc: 0.871372
start validation


Loss: 0.40 #nan->0: 100%|██████████| 172/172 [01:20<00:00,  2.13it/s]


validation loss: 0.4940530061721802
validation accuracy: 0.7943313953488372
Save new model!- 17/10 h25:43
--------------------
* Epoch 18/100


Loss: 0.39 #nan->0:  35%|███▍      | 240/689 [03:04<05:44,  1.30it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_dropout=0.5_healthy-unhealthy_hubert_base_lr=1e-05_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=17)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 18/100


Loss: 0.47 #nan->0: 100%|██████████| 689/689 [08:15<00:00,  1.39it/s]


loss: 0.283277
acc: 0.883164
start validation


Loss: 0.42 #nan->0: 100%|██████████| 172/172 [01:20<00:00,  2.15it/s]


validation loss: 0.5456808805465698
validation accuracy: 0.7863372093023255
Save new model!- 18/10 h10:51
--------------------
* Epoch 19/100


...

validation loss: 0.6557899713516235
validation accuracy: 0.7972383720930233
--------------------
* Epoch 27/100


Loss: 0.41 #nan->0: 100%|██████████| 689/689 [08:16<00:00,  1.39it/s]


loss: 0.188268
acc: 0.925980
start validation


Loss: 0.28 #nan->0: 100%|██████████| 172/172 [01:21<00:00,  2.11it/s]


validation loss: 0.666949987411499
validation accuracy: 0.8132267441860465
--------------------


-----------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.65 #nan->0: 100%|██████████| 689/689 [08:50<00:00,  1.30it/s]


loss: 0.714450
acc: 0.504898
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [01:19<00:00,  2.18it/s]


validation loss: 0.6971395611763
validation accuracy: 0.5174418604651163
Save new model!- 16/10 h19:59
--------------------
* Epoch 2/100


...

Loss: 0.70 #nan->0: 100%|██████████| 689/689 [08:46<00:00,  1.31it/s]


loss: 0.705280
acc: 0.507257
start validation


Loss: 0.70 #nan->0: 100%|██████████| 172/172 [01:18<00:00,  2.18it/s]


validation loss: 0.6948500275611877
validation accuracy: 0.48255813953488375
--------------------
* Epoch 6/100


Loss: 0.79 #nan->0:  13%|█▎        | 93/689 [01:12<07:32,  1.32it/s]

----------------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.78 #nan->0: 100%|██████████| 689/689 [08:22<00:00,  1.37it/s]


loss: 0.688122
acc: 0.547714
start validation


Loss: 0.55 #nan->0: 100%|██████████| 172/172 [00:56<00:00,  3.04it/s]


validation loss: 0.6821467876434326
validation accuracy: 0.5625
Save new model!- 16/10 h21:15
--------------------
* Epoch 2/100


...

validation loss: 0.5598865151405334
validation accuracy: 0.7659883720930233
--------------------
* Epoch 13/100


Loss: 0.22 #nan->0: 100%|██████████| 689/689 [08:24<00:00,  1.37it/s]


loss: 0.389321
acc: 0.831821
start validation


Loss: 0.41 #nan->0: 100%|██████████| 172/172 [00:57<00:00,  3.01it/s]


validation loss: 0.5200614333152771
validation accuracy: 0.78125
--------------------


---------------------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.71 #nan->0: 100%|██████████| 689/689 [08:22<00:00,  1.37it/s]


loss: 0.706455
acc: 0.509797
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:56<00:00,  3.02it/s]


validation loss: 0.6840344071388245
validation accuracy: 0.5334302325581395
Save new model!- 16/10 h23:17
--------------------
* Epoch 2/100


Loss: 0.72 #nan->0: 100%|██████████| 689/689 [08:25<00:00,  1.36it/s]


loss: 0.694053
acc: 0.526669
start validation


Loss: 0.65 #nan->0: 100%|██████████| 172/172 [00:58<00:00,  2.92it/s]


validation loss: 0.665809690952301
validation accuracy: 0.623546511627907
Save new model!- 16/10 h23:27
--------------------
* Epoch 3/100


Loss: 0.63 #nan->0:  64%|██████▎   | 439/689 [05:21<03:03,  1.36it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=2)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 3/100


Loss: 0.72 #nan->0: 100%|██████████| 689/689 [08:34<00:00,  1.34it/s]


loss: 0.683603
acc: 0.555152
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [01:02<00:00,  2.77it/s]


validation loss: 0.6512445211410522
validation accuracy: 0.6366279069767442
Save new model!- 17/10 h2:29
--------------------
* Epoch 4/100


...

Loss: 0.59 #nan->0: 100%|██████████| 689/689 [08:33<00:00,  1.34it/s]


loss: 0.565597
acc: 0.715167
start validation


Loss: 0.48 #nan->0: 100%|██████████| 172/172 [01:02<00:00,  2.76it/s]


validation loss: 0.6101799607276917
validation accuracy: 0.7100290697674418
--------------------
* Epoch 26/100


Loss: 0.69 #nan->0:  41%|████      | 280/689 [03:30<05:02,  1.35it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=24)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 25/100


Loss: 0.63 #nan->0: 100%|██████████| 689/689 [08:23<00:00,  1.37it/s]


loss: 0.561276
acc: 0.720247
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [01:00<00:00,  2.83it/s]


validation loss: 0.5476035475730896
validation accuracy: 0.7318313953488372
Save new model!- 17/10 h11:25
--------------------
* Epoch 26/100


...

validation loss: 0.5367612838745117
validation accuracy: 0.7536337209302325
--------------------
* Epoch 39/100


Loss: 0.52 #nan->0: 100%|██████████| 689/689 [08:24<00:00,  1.37it/s]


loss: 0.518794
acc: 0.755443
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [01:00<00:00,  2.85it/s]


validation loss: 0.5441237092018127
validation accuracy: 0.7412790697674418
--------------------


In [ ]:
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=39)

* Epoch 40/100


Loss: 0.41 #nan->0: 100%|██████████| 689/689 [08:14<00:00,  1.39it/s]


loss: 0.504682
acc: 0.763062
start validation


Loss: 0.41 #nan->0: 100%|██████████| 172/172 [01:00<00:00,  2.85it/s]


validation loss: 0.5542766451835632
validation accuracy: 0.7492732558139535
Save new model!- 17/10 h14:39
--------------------
* Epoch 41/100


Loss: 0.77 #nan->0:  70%|███████   | 485/689 [05:48<02:28,  1.37it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=40)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 41/100


Loss: 1.18 #nan->0: 100%|██████████| 689/689 [08:49<00:00,  1.30it/s]


loss: 0.513114
acc: 0.763062
start validation


Loss: 0.47 #nan->0: 100%|██████████| 172/172 [01:02<00:00,  2.75it/s]


validation loss: 0.5387439131736755
validation accuracy: 0.7463662790697675
Save new model!- 17/10 h15:17
--------------------
* Epoch 42/100


...

validation loss: 0.564294695854187
validation accuracy: 0.7311046511627907
--------------------
* Epoch 46/100


Loss: 0.59 #nan->0: 100%|██████████| 689/689 [08:48<00:00,  1.30it/s]


loss: 0.494474
acc: 0.770319
start validation


Loss: 0.53 #nan->0: 100%|██████████| 172/172 [01:02<00:00,  2.77it/s]

validation loss: 0.5544770359992981
validation accuracy: 0.7601744186046512
--------------------


In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-_bandPass_healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=41)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 42/100


Loss: 0.24 #nan->0: 100%|██████████| 689/689 [08:50<00:00,  1.30it/s]


loss: 0.507996
acc: 0.765239
start validation


Loss: 0.41 #nan->0: 100%|██████████| 172/172 [01:19<00:00,  2.18it/s]


validation loss: 0.5423929691314697
validation accuracy: 0.7507267441860465
Save new model!- 17/10 h17:22
--------------------
* Epoch 43/100


...

validation loss: 0.5473501682281494
validation accuracy: 0.7572674418604651
--------------------
* Epoch 49/100


Loss: 0.68 #nan->0: 100%|██████████| 689/689 [08:43<00:00,  1.32it/s]


loss: 0.487634
acc: 0.774492
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [01:19<00:00,  2.17it/s]


validation loss: 0.568111002445221
validation accuracy: 0.7420058139534884
--------------------


------------------------

----------------------------------

**Hubert-base**

In [ ]:
freq=16
duration=5
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 2
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.89 #nan->0: 100%|██████████| 689/689 [13:51<00:00,  1.21s/it]


loss: 0.710476
acc: 0.521226
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 0.7058281898498535
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.703395
acc: 0.514514
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.30it/s]


validation loss: 0.7028549313545227
validation accuracy: 0.5174418604651163
--------------------
* Epoch 6/100


Loss: 0.64 #nan->0:  43%|████▎     | 293/689 [05:54<07:58,  1.21s/it]


KeyboardInterrupt: ignored

--------------------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.81 #nan->0: 100%|██████████| 689/689 [13:53<00:00,  1.21s/it]


loss: 0.679538
acc: 0.569122
start validation


Loss: 0.53 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.30it/s]


validation loss: 0.6321563124656677
validation accuracy: 0.6635174418604651
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.48 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.26it/s]


validation loss: 0.5335274338722229
validation accuracy: 0.7645348837209303
Save new model!
--------------------
* Epoch 7/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [14:08<00:00,  1.23s/it]


loss: 0.527687
acc: 0.757801
start validation


Loss: 0.41 #nan->0:  50%|█████     | 86/172 [00:38<00:37,  2.27it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-05_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=6)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 7/100


Loss: 0.20 #nan->0: 100%|██████████| 689/689 [12:57<00:00,  1.13s/it]


loss: 0.530997
acc: 0.750726
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.6523984670639038
validation accuracy: 0.7013081395348837
Save new model!
--------------------
* Epoch 8/100


...

validation loss: 0.558337926864624
validation accuracy: 0.7630813953488372
--------------------
* Epoch 13/100


Loss: 0.22 #nan->0: 100%|██████████| 689/689 [12:58<00:00,  1.13s/it]


loss: 0.396605
acc: 0.833273
start validation


Loss: 0.73 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.5532245635986328
validation accuracy: 0.7718023255813954
--------------------


In [ ]:
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=13)

* Epoch 14/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [12:59<00:00,  1.13s/it]


loss: 0.375894
acc: 0.843433
start validation


Loss: 0.79 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.53944331407547
validation accuracy: 0.7696220930232558
Save new model!
--------------------
* Epoch 15/100


...

validation loss: 0.5868187546730042
validation accuracy: 0.7776162790697675
--------------------
* Epoch 19/100


Loss: 0.37 #nan->0: 100%|██████████| 689/689 [13:01<00:00,  1.13s/it]


loss: 0.285427
acc: 0.882075
start validation


Loss: 0.54 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.6579572558403015
validation accuracy: 0.7572674418604651
--------------------


In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-05_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=14)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 15/100


Loss: 0.46 #nan->0: 100%|██████████| 689/689 [14:16<00:00,  1.24s/it]


loss: 0.350345
acc: 0.850145
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.24it/s]


validation loss: 0.5940243601799011
validation accuracy: 0.752906976744186
Save new model!
--------------------
* Epoch 16/100


...

validation loss: 0.6205856204032898
validation accuracy: 0.7936046511627907
--------------------
* Epoch 24/100


Loss: 0.69 #nan->0: 100%|██████████| 689/689 [14:19<00:00,  1.25s/it]


loss: 0.230461
acc: 0.911466
start validation


Loss: 1.26 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.23it/s]


validation loss: 0.6655162572860718
validation accuracy: 0.7768895348837209
--------------------


------------------------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.63 #nan->0: 100%|██████████| 689/689 [14:21<00:00,  1.25s/it]


loss: 0.696065
acc: 0.523222
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [01:17<00:00,  2.23it/s]


validation loss: 0.6793243885040283
validation accuracy: 0.529796511627907
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.68 #nan->0: 100%|██████████| 689/689 [14:20<00:00,  1.25s/it]


loss: 0.687447
acc: 0.543179
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [01:17<00:00,  2.23it/s]


validation loss: 0.6570813655853271
validation accuracy: 0.6242732558139535
Save new model!
--------------------
* Epoch 4/100


Loss: 0.63 #nan->0:  33%|███▎      | 230/689 [04:47<09:26,  1.23s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=3)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 4/100


Loss: 0.65 #nan->0: 100%|██████████| 689/689 [13:56<00:00,  1.21s/it]


loss: 0.691061
acc: 0.535196
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.30it/s]


validation loss: 0.6696105003356934
validation accuracy: 0.5755813953488372
Save new model!
--------------------
* Epoch 5/100


...

Loss: 0.56 #nan->0: 100%|██████████| 689/689 [13:49<00:00,  1.20s/it]


loss: 0.620401
acc: 0.663462
start validation


Loss: 0.45 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 0.5957742929458618
validation accuracy: 0.7143895348837209
--------------------
* Epoch 16/100


Loss: 0.60 #nan->0:  60%|██████    | 416/689 [08:21<05:38,  1.24s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=14)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 15/100


Loss: 0.58 #nan->0: 100%|██████████| 689/689 [14:03<00:00,  1.22s/it]


loss: 0.621043
acc: 0.663462
start validation


Loss: 0.47 #nan->0: 100%|██████████| 172/172 [01:15<00:00,  2.28it/s]


validation loss: 0.5870137214660645
validation accuracy: 0.7071220930232558
Save new model!
--------------------
* Epoch 16/100


...

validation loss: 0.6031874418258667
validation accuracy: 0.7100290697674418
--------------------
* Epoch 23/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [14:07<00:00,  1.23s/it]


loss: 0.576118
acc: 0.711176
start validation


Loss: 0.41 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.25it/s]


validation loss: 0.5792240500450134
validation accuracy: 0.7274709302325582
--------------------


In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=18)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 19/100


Loss: 0.70 #nan->0: 100%|██████████| 689/689 [13:17<00:00,  1.16s/it]


loss: 0.692260
acc: 0.534107
start validation


Loss: 0.62 #nan->0: 100%|██████████| 172/172 [01:10<00:00,  2.43it/s]


validation loss: 0.6614434719085693
validation accuracy: 0.6068313953488372
Save new model!
--------------------
* Epoch 20/100


...

Loss: 0.37 #nan->0: 100%|██████████| 172/172 [01:11<00:00,  2.42it/s]


validation loss: 0.6081940531730652
validation accuracy: 0.6984011627906976
--------------------
* Epoch 31/100


Loss: 0.57 #nan->0: 100%|██████████| 689/689 [13:15<00:00,  1.15s/it]


loss: 0.621541
acc: 0.662917
start validation


Loss: 0.63 #nan->0:  97%|█████████▋| 166/172 [01:08<00:02,  2.43it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=28)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 29/100


Loss: 0.65 #nan->0: 100%|██████████| 689/689 [13:00<00:00,  1.13s/it]


loss: 0.630975
acc: 0.653120
start validation


Loss: 0.40 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.596417248249054
validation accuracy: 0.6998546511627907
Save new model!
--------------------
* Epoch 30/100


...

validation loss: 0.6087169647216797
validation accuracy: 0.7013081395348837
--------------------
* Epoch 38/100


Loss: 0.88 #nan->0: 100%|██████████| 689/689 [13:00<00:00,  1.13s/it]


loss: 0.591064
acc: 0.697206
start validation


Loss: 0.35 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.46it/s]


validation loss: 0.597668468952179
validation accuracy: 0.7093023255813954
--------------------


In [ ]:
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=38)

* Epoch 39/100


Loss: 0.55 #nan->0: 100%|██████████| 689/689 [12:58<00:00,  1.13s/it]


loss: 0.586594
acc: 0.699202
start validation


Loss: 0.34 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.5567843914031982
validation accuracy: 0.7311046511627907
Save new model!
--------------------
* Epoch 40/100


...

Loss: 0.75 #nan->0: 100%|██████████| 689/689 [13:00<00:00,  1.13s/it]


loss: 0.566793
acc: 0.718433
start validation


Loss: 0.36 #nan->0: 100%|██████████| 172/172 [01:09<00:00,  2.47it/s]


validation loss: 0.6044736504554749
validation accuracy: 0.6976744186046512
--------------------
* Epoch 44/100


Loss: 0.51 #nan->0:  29%|██▉       | 203/689 [03:51<09:08,  1.13s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-06_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=41)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 42/100


Loss: 0.60 #nan->0: 100%|██████████| 689/689 [14:13<00:00,  1.24s/it]


loss: 0.575218
acc: 0.712264
start validation


Loss: 0.35 #nan->0: 100%|██████████| 172/172 [01:15<00:00,  2.28it/s]


validation loss: 0.5694136619567871
validation accuracy: 0.7223837209302325
Save new model!
--------------------
* Epoch 43/100


...

validation loss: 0.568938136100769
validation accuracy: 0.720203488372093
--------------------
* Epoch 48/100


Loss: 0.48 #nan->0: 100%|██████████| 689/689 [14:07<00:00,  1.23s/it]


loss: 0.540918
acc: 0.741292
start validation


Loss: 0.30 #nan->0: 100%|██████████| 172/172 [01:15<00:00,  2.28it/s]


validation loss: 0.5718799233436584
validation accuracy: 0.7180232558139535
--------------------


----------------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-05_77%-75%_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], '1e-5+1e-6', last_epoch=8)
#16mhz 5secondi tanh dropout=0.7 batch=8 wav2vec2
#DALLA 8° EPOCA: 77%-75%
#DALLA 20ESIMA EPOCA PASSO A LEARNING RATE=1E-6

* Epoch 9/100


Loss: 0.80 #nan->0: 100%|██████████| 689/689 [14:16<00:00,  1.24s/it]


loss: 0.460938
acc: 0.800798
start validation


Loss: 0.51 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.25it/s]


validation loss: 0.526678204536438
validation accuracy: 0.7587209302325582
Save new model!- 16/10 h13:17
--------------------
* Epoch 10/100


...

validation loss: 0.5532503724098206
validation accuracy: 0.7594476744186046
--------------------
* Epoch 14/100


Loss: 0.39 #nan->0: 100%|██████████| 689/689 [14:14<00:00,  1.24s/it]


loss: 0.415097
acc: 0.822569
start validation


Loss: 0.77 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.24it/s]


validation loss: 0.5641370415687561
validation accuracy: 0.7558139534883721
--------------------


In [ ]:
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], '1e-5+1e-6', last_epoch=14)


* Epoch 15/100


Loss: 0.46 #nan->0: 100%|██████████| 689/689 [14:16<00:00,  1.24s/it]


loss: 0.407526
acc: 0.828012
start validation


Loss: 0.80 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.25it/s]


validation loss: 0.5687248110771179
validation accuracy: 0.7594476744186046
Save new model!- 16/10 h15:5
--------------------
* Epoch 16/100


...

Loss: 0.75 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.25it/s]


validation loss: 0.5744166374206543
validation accuracy: 0.7536337209302325
--------------------
* Epoch 17/100


Loss: 0.51 #nan->0: 100%|██████████| 689/689 [14:16<00:00,  1.24s/it]


loss: 0.403519
acc: 0.832003
start validation


Loss: 0.65 #nan->0:   4%|▍         | 7/172 [00:03<01:15,  2.20it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_base_lr=1e-5+1e-6_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=15)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 16/100


Loss: 0.15 #nan->0: 100%|██████████| 689/689 [13:11<00:00,  1.15s/it]


loss: 0.412557
acc: 0.824202
start validation


Loss: 0.71 #nan->0: 100%|██████████| 172/172 [01:10<00:00,  2.43it/s]


validation loss: 0.5625250339508057
validation accuracy: 0.7521802325581395
Save new model!- 16/10 h16:30
--------------------
* Epoch 17/100


...

validation loss: 0.5837541222572327
validation accuracy: 0.7652616279069767
--------------------
* Epoch 23/100


Loss: 0.16 #nan->0: 100%|██████████| 689/689 [13:14<00:00,  1.15s/it]


loss: 0.380769
acc: 0.838897
start validation


Loss: 0.76 #nan->0: 100%|██████████| 172/172 [01:11<00:00,  2.41it/s]

validation loss: 0.5838091373443604
validation accuracy: 0.7594476744186046
--------------------


In [ ]:
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=23)

* Epoch 24/100


Loss: 0.71 #nan->0: 100%|██████████| 689/689 [13:15<00:00,  1.15s/it]


loss: 0.377839
acc: 0.841074
start validation


Loss: 0.57 #nan->0: 100%|██████████| 172/172 [01:10<00:00,  2.43it/s]


validation loss: 0.5792063474655151
validation accuracy: 0.7616279069767442
Save new model!- 16/10 h18:46
--------------------
* Epoch 25/100


...

Loss: 0.30 #nan->0: 100%|██████████| 689/689 [13:19<00:00,  1.16s/it]


loss: 0.367707
acc: 0.850871
start validation


Loss: 0.52 #nan->0: 100%|██████████| 172/172 [01:10<00:00,  2.43it/s]


validation loss: 0.5762501358985901
validation accuracy: 0.7609011627906976
--------------------
* Epoch 27/100


Loss: 0.67 #nan->0:  53%|█████▎    | 366/689 [07:04<06:28,  1.20s/it]

-------------------------

--------------------------

#### large

In [ ]:
freq=16
duration=5
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 6
num_workers = 2
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-large-ls960-ft")

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.85 #nan->0: 100%|██████████| 919/919 [34:16<00:00,  2.24s/it]


loss: 0.705847
acc: 0.517592
start validation


Loss: 0.71 #nan->0: 100%|██████████| 230/230 [03:02<00:00,  1.26it/s]


validation loss: 0.6956894397735596
validation accuracy: 0.5173913043478261
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.73 #nan->0: 100%|██████████| 230/230 [03:03<00:00,  1.25it/s]


validation loss: 0.6239804625511169
validation accuracy: 0.6869565217391305
--------------------
* Epoch 9/100


Loss: 0.37 #nan->0: 100%|██████████| 919/919 [34:36<00:00,  2.26s/it]


loss: 0.588190
acc: 0.706202
start validation


Loss: 1.14 #nan->0:   0%|          | 1/230 [00:01<04:11,  1.10s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_lr=0.0001_batch=6_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=7)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 8/100


Loss: 0.57 #nan->0: 100%|██████████| 919/919 [34:06<00:00,  2.23s/it]


loss: 0.601907
acc: 0.689518
start validation


Loss: 0.46 #nan->0: 100%|██████████| 230/230 [03:00<00:00,  1.28it/s]


validation loss: 0.5831150412559509
validation accuracy: 0.6884057971014492
Save new model!
--------------------
* Epoch 9/100


...

Loss: 1.10 #nan->0: 100%|██████████| 919/919 [34:34<00:00,  2.26s/it]


loss: 0.525505
acc: 0.751542
start validation


Loss: 0.83 #nan->0: 100%|██████████| 230/230 [03:03<00:00,  1.25it/s]


validation loss: 0.5971018075942993
validation accuracy: 0.7195652173913043
--------------------
* Epoch 13/100


Loss: 0.17 #nan->0:  56%|█████▌    | 514/919 [19:26<14:55,  2.21s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_lr=0.0001_batch=6_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=11)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 12/100


Loss: 0.17 #nan->0: 100%|██████████| 919/919 [35:16<00:00,  2.30s/it]


loss: 0.526028
acc: 0.752267
start validation


Loss: 0.75 #nan->0: 100%|██████████| 230/230 [03:07<00:00,  1.23it/s]


validation loss: 0.5756949782371521
validation accuracy: 0.7297101449275363
Save new model!
--------------------
* Epoch 13/100


...

Loss: 0.27 #nan->0: 100%|██████████| 919/919 [35:14<00:00,  2.30s/it]


loss: 0.427189
acc: 0.810845
start validation


Loss: 0.61 #nan->0: 100%|██████████| 230/230 [03:04<00:00,  1.25it/s]


validation loss: 0.5650206208229065
validation accuracy: 0.7391304347826086
--------------------
* Epoch 17/100


Loss: 0.23 #nan->0:  49%|████▉     | 454/919 [17:19<18:18,  2.36s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_lr=0.0001_batch=6_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=15)
#16mhz 5secondi tanh dropout=0.7 batch=8 wav2vec2

* Epoch 16/100


Loss: 0.46 #nan->0: 100%|██████████| 919/919 [35:08<00:00,  2.29s/it]


loss: 0.423785
acc: 0.814291
start validation


Loss: 0.87 #nan->0: 100%|██████████| 230/230 [03:06<00:00,  1.24it/s]


validation loss: 0.6641530394554138
validation accuracy: 0.7289855072463768
Save new model!
--------------------
* Epoch 17/100


...

Loss: 0.36 #nan->0: 100%|██████████| 919/919 [35:32<00:00,  2.32s/it]


loss: 0.380182
acc: 0.835328
start validation


Loss: 0.17 #nan->0: 100%|██████████| 230/230 [03:06<00:00,  1.23it/s]


validation loss: 0.6420225501060486
validation accuracy: 0.7123188405797102
Save new model!
--------------------
* Epoch 19/100


Loss: 0.57 #nan->0:  15%|█▍        | 136/919 [05:16<29:15,  2.24s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Hubert/OK-healthy-unhealthy_hubert_lr=0.0001_batch=6_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=18)
#16mhz 5secondi tanh dropout=0.7 batch=8 hubert

* Epoch 19/100


Loss: 0.57 #nan->0: 100%|██████████| 919/919 [34:39<00:00,  2.26s/it]


loss: 0.361081
acc: 0.846028
start validation


Loss: 0.70 #nan->0: 100%|██████████| 230/230 [03:03<00:00,  1.26it/s]


validation loss: 0.7209454774856567
validation accuracy: 0.741304347826087
Save new model!
--------------------
* Epoch 20/100


...

Loss: 0.75 #nan->0: 100%|██████████| 919/919 [34:49<00:00,  2.27s/it]


loss: 0.288110
acc: 0.876496
start validation


Loss: 0.90 #nan->0: 100%|██████████| 230/230 [03:03<00:00,  1.26it/s]


validation loss: 0.698552668094635
validation accuracy: 0.7485507246376811
--------------------
* Epoch 24/100


Loss: 0.11 #nan->0:  17%|█▋        | 159/919 [06:01<29:19,  2.31s/it]

---------------------------


In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-large-ls960-ft")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

---------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-large-ls960-ft")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =3e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

-----------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-large-ls960-ft")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =8e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

---------------------

In [ ]:
DEVICE = 'cuda'
m = AutoModel.from_pretrained("facebook/hubert-large-ls960-ft")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =5e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

---------------------

-------------------

#### prova

In [ ]:
print('ciao')

ciao


In [ ]:
freq=44
duration=15
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 4
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 15secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.55 #nan->0:  92%|█████████▏| 1265/1379 [13:00<01:10,  1.62it/s]


RuntimeError: ignored

---------------

In [ ]:
freq=4
duration=3
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 4secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.72 #nan->0: 100%|██████████| 689/689 [02:00<00:00,  5.72it/s]


loss: 0.695234
acc: 0.554064
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.31it/s]


validation loss: 0.624635636806488
validation accuracy: 0.6773255813953488
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.9656369686126709
validation accuracy: 0.7042151162790697
--------------------
* Epoch 8/100


Loss: 0.04 #nan->0: 100%|██████████| 689/689 [01:11<00:00,  9.59it/s]


loss: 0.171909
acc: 0.935051
start validation


Loss: 1.06 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.35it/s]


validation loss: 1.1245341300964355
validation accuracy: 0.7194767441860465
--------------------


-----------------

-----------------------------

In [ ]:
freq=16
duration=4
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [04:52<00:00,  2.35it/s]


loss: 0.679650
acc: 0.572206
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [00:30<00:00,  5.62it/s]


validation loss: 0.6456491947174072
validation accuracy: 0.6017441860465116
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.8380417227745056
validation accuracy: 0.717296511627907
--------------------
* Epoch 8/100


Loss: 0.29 #nan->0: 100%|██████████| 689/689 [03:59<00:00,  2.87it/s]


loss: 0.154695
acc: 0.938498
start validation


Loss: 1.52 #nan->0: 100%|██████████| 172/172 [00:30<00:00,  5.55it/s]

validation loss: 0.8778241276741028
validation accuracy: 0.7463662790697675
--------------------


In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [03:59<00:00,  2.87it/s]


loss: 0.697113
acc: 0.538643
start validation


Loss: 0.65 #nan->0: 100%|██████████| 172/172 [00:31<00:00,  5.54it/s]


validation loss: 0.6732527017593384
validation accuracy: 0.5901162790697675
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.568950355052948
validation accuracy: 0.7303779069767442
--------------------
* Epoch 12/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [03:56<00:00,  2.91it/s]


loss: 0.434763
acc: 0.797896
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [00:31<00:00,  5.53it/s]

validation loss: 0.563697338104248
validation accuracy: 0.7325581395348837
--------------------


In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =3e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [03:57<00:00,  2.90it/s]


loss: 0.696829
acc: 0.535015
start validation


Loss: 0.62 #nan->0: 100%|██████████| 172/172 [00:30<00:00,  5.59it/s]


validation loss: 0.6727363467216492
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6921599507331848
validation accuracy: 0.7063953488372093
--------------------
* Epoch 10/100


Loss: 0.14 #nan->0: 100%|██████████| 689/689 [03:57<00:00,  2.90it/s]


loss: 0.252872
acc: 0.896045
start validation


Loss: 1.28 #nan->0: 100%|██████████| 172/172 [00:30<00:00,  5.56it/s]

validation loss: 0.9168079495429993
validation accuracy: 0.6925872093023255
--------------------


------------------------------

------------------------

In [ ]:
# model = Classifier(m)

In [ ]:
# list2 =[0,1,2,3,4,5]
# # list2 = list2.reshape(3,2)
# tmp = torch.tensor([e+1 for e in list2])
# tmp = tmp.reshape(3,2)
# tmp.reshape(1,6)[0].tolist()
# tmp2 = torch.tensor([e+1 for e in tmp.reshape(1,6)[0].tolist()])
# tmp2

tensor([2, 3, 4, 5, 6, 7])

In [ ]:
# from tqdm import tqdm
# import math
# from torch import autograd


# def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, max_stable=100):
#     best_val_loss = 1e9
#     counter = 0
#     nan=False
#     for epoch in range(epochs):
#         correct_predictions = 0
#         count=0
#         #tic = time()
#         print('* Epoch %d/%d' % (epoch+1, epochs))

#         avg_loss = 0
#         model.train()  # train mode
#         progress = tqdm(data_tr)
#         for batch in progress:
#             loss = 0
#             # data to device
#             X_batch, Y_batch = batch["representation"], batch["target"]
#             # print(X_batch.shape)
#             X_batch = X_batch.to(DEVICE)
#             Y_batch = Y_batch.to(DEVICE)
#             # with torch.autograd.detect_anomaly():
# #                     outputs = GB(inputs)
# #                     loss = LOSS(outputs, labels)
# # #                    loss.register_hook(lambda grad : print(grad))
# #                     loss.backward()
#             # set parameter gradients to zero
#             opt.zero_grad()
#             # forward
#             Y_pred = model(X_batch)
#             # #remove NAN
            
#             # indici = np.argwhere(np.isnan(Y_pred))[0].unique().numpy())#indici

#             # Y_pred = Y_pred[np.isfinite(Y_pred)]
#             # Y_batch
#             # print(X_batch)
#             # print(Y_pred)
#             # print(Y_batch)
#             # Y_pred = torch.tensor([e+1e-6 for e in Y_pred.reshape(1,16)[0].tolist()])
#             # Y_pred = Y_pred.reshape(8,2)
#             _, preds = torch.max(Y_pred, dim=1)
#             # loss = loss_fn(outputs[0], targets)
#             # loss = outputs[0]
#             # print(preds)
#             # print(Y_batch)
#             preds = preds.to(DEVICE)
#             Y_pred = Y_pred.to(DEVICE)
            

#             # print('---------')
#             # print('Y_pred')
#             # print(Y_pred)
            
#             # print('preds')
#             # print(preds)
#             # print('X_batch')
#             # print(X_batch)
#             # print('Y_batch')
#             # print(Y_batch)
#             # print('---------')

#             loss = loss_fn(Y_pred, Y_batch)# forward-pass
#             nan = math.isnan(loss)

#             if nan:
#               count=count+1
#               nan=False
#               continue
#               # print('---------')
#               # print('X_batch')
#               # print(X_batch.isnan().any())
#               # print('Y_pred')
#               # print(Y_pred)
#               # print(Y_pred.isnan().any())
              
#               # print('preds')
#               # print(preds.isnan().any())
              
#               # print('Y_batch')
#               # print(Y_batch.isnan().any())
#               # 
#               # print('---------')
#             # if math.isnan(loss):
#             #   print('ok')
#             # nan = True if loss is np.nan else False
#             # print('-------')
#             # print(loss)
#             # print('-------')
#             correct_predictions += torch.sum(preds == Y_batch)
#             progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
#             loss.backward()  # backward-pass
#             # torch.nn.utils.clip_grad_norm_(model.parameters(), 25)
#             opt.step()  # update weights
#             if not scheduler is None:
#                 scheduler.step()
#             # calculate loss to show the user
#             avg_loss += loss
#         avg_loss /= (len(data_tr)-(batch_size*count))
#       #  toc = time()
#         avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
#         print('loss: %f' % avg_loss)
#         print('acc: %f' % avg_acc)
#         # show intermediate results


#         model.eval()  # testing mode
#         val_loss = 0
#         val_acc = 0
#         print("start validation")
#         count_loss_val=0
#         progress_val = tqdm(data_val)
#         correct_predictions_val = 0
#         for v_b in progress_val:
#             X_val, Y_val = v_b["representation"], v_b["target"]
#             # X_val = X_val.to(DEVICE)#
#             # Y_val = Y_val.to(DEVICE)#
#             Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
#             _, preds_val = torch.max(Y_hat, dim=1)
#             # preds_val = preds_val.to(DEVICE)#

#             loss_val = loss_fn(Y_hat, Y_val)
#             nan_loss = math.isnan(loss_val) #check loss==nan
#             if nan_loss:
#               count_loss_val = count_loss_val+1
#               nan_loss = False
#               continue
#             #se trova un nan, salta tutto il batch
#             correct_predictions_val += torch.sum(preds_val == Y_val)
#             progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
#             val_loss += loss_val

#         val_loss /= (len(data_val)-(batch_size*count_loss_val))
#         val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
#         print( f"validation loss: {val_loss}")
#         print( f"validation accuracy: {val_acc}")
#         if val_loss <= best_val_loss and val_loss > 0:
#             counter = 0
#             print("Save new model!")
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), '/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-4_batch=12_16mhz_dropout=07.bin')
#         else:
#             counter += 1
#         print("--------------------")
#         if counter == max_stable:
#             break

In [ ]:
# before running this part it is neccessary to run at least 1 batch through the model in order for inputs
# to adapt
# model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/3MHz-healthy-unhealthy_aalbert.bin'))

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12

* Epoch 1/100


Loss: 0.67 #nan->0: 100%|██████████| 459/459 [03:53<00:00,  1.97it/s]


loss: 0.690515
acc: 0.555556
start validation


Loss: 0.66 #nan->0: 100%|██████████| 115/115 [00:36<00:00,  3.15it/s]


validation loss: 0.6301531195640564
validation accuracy: 0.6536231884057971
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.200593
acc: 0.921932
start validation


Loss: 1.43 #nan->0: 100%|██████████| 115/115 [00:37<00:00,  3.08it/s]


validation loss: 0.7797514796257019
validation accuracy: 0.7340579710144928
--------------------
* Epoch 9/100


Loss: 0.05 #nan->0:  61%|██████    | 280/459 [01:54<01:13,  2.45it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi tanh dropout=0.7 batch=12

* Epoch 1/100


Loss: 0.92 #nan->0: 100%|██████████| 459/459 [01:52<00:00,  4.09it/s]


loss: 0.686976
acc: 0.555919
start validation


Loss: 0.73 #nan->0: 100%|██████████| 115/115 [00:13<00:00,  8.31it/s]


validation loss: 0.6566121578216553
validation accuracy: 0.6028985507246377
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.022828
acc: 0.993101
start validation


Loss: 1.87 #nan->0: 100%|██████████| 115/115 [00:13<00:00,  8.34it/s]


validation loss: 1.5146046876907349
validation accuracy: 0.7268115942028985
--------------------
* Epoch 41/100


Loss: 0.00 #nan->0:  12%|█▏        | 54/459 [00:09<01:09,  5.84it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [02:01<00:00,  5.69it/s]


loss: 0.682495
acc: 0.562409
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.81it/s]


validation loss: 0.6448449492454529
validation accuracy: 0.6329941860465116
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.8822665214538574
validation accuracy: 0.7143895348837209
--------------------
* Epoch 31/100


Loss: 0.00 #nan->0: 100%|██████████| 689/689 [01:10<00:00,  9.83it/s]


loss: 0.032987
acc: 0.989478
start validation


Loss: 0.30 #nan->0:  25%|██▌       | 43/172 [00:05<00:15,  8.39it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 6secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [02:12<00:00,  5.21it/s]


loss: 0.686376
acc: 0.561865
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.78it/s]


validation loss: 0.63966965675354
validation accuracy: 0.6329941860465116
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.109003
acc: 0.960631
start validation


Loss: 0.28 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.13it/s]


validation loss: 1.0214635133743286
validation accuracy: 0.7434593023255814
--------------------
* Epoch 11/100


  0%|          | 0/689 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#5mhz 6secondi relu dropout=0.5

* Epoch 1/100


Loss: 0.59 #nan->0: 100%|██████████| 689/689 [02:26<00:00,  4.71it/s]


loss: 0.674037
acc: 0.577286
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:17<00:00,  9.86it/s]


validation loss: 0.6237277388572693
validation accuracy: 0.6686046511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.191441
acc: 0.925435
start validation


Loss: 0.45 #nan->0: 100%|██████████| 172/172 [00:17<00:00,  9.69it/s]


validation loss: 0.752937376499176
validation accuracy: 0.7238372093023255
--------------------
* Epoch 8/100


Loss: 0.09 #nan->0:  14%|█▍        | 95/689 [00:14<01:33,  6.34it/s]


KeyboardInterrupt: ignored

In [ ]:
#quello prima dalla 4° epoca (74%, 71%) ma con lr=1e-5
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-4_relu_6sec_5mhz.bin'))
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#5mhz 6secondi relu dropout=0.5 lr=1e-5

* Epoch 1/100


Loss: 0.32 #nan->0: 100%|██████████| 689/689 [01:34<00:00,  7.32it/s]


loss: 0.378235
acc: 0.831640
start validation


Loss: 0.54 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.22it/s]


validation loss: 0.6146918535232544
validation accuracy: 0.7311046511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.132549
acc: 0.953193
start validation


Loss: 0.59 #nan->0: 100%|██████████| 172/172 [00:17<00:00, 10.04it/s]


validation loss: 0.9501494765281677
validation accuracy: 0.7194767441860465
--------------------
* Epoch 13/100


Loss: 0.05 #nan->0:  94%|█████████▍| 651/689 [01:29<00:05,  7.24it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.5

* Epoch 1/100


Loss: 0.69 #nan->0: 100%|██████████| 689/689 [01:11<00:00,  9.69it/s]


loss: 0.686886
acc: 0.546081
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.63it/s]


validation loss: 0.6617732048034668
validation accuracy: 0.5901162790697675
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.205552
acc: 0.916364
start validation


Loss: 0.18 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 10.95it/s]


validation loss: 0.8970562219619751
validation accuracy: 0.7231104651162791
--------------------
* Epoch 24/100


Loss: 0.28 #nan->0:   6%|▌         | 43/689 [00:06<01:38,  6.55it/s]


KeyboardInterrupt: ignored

In [ ]:
#quello prima dalla 7° epoca (71%, 69%) ma con lr=1e-6
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-5_relu_dropout=0.5.bin'))
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.5 lr=1e-6

* Epoch 1/100


Loss: 0.93 #nan->0: 100%|██████████| 689/689 [01:10<00:00,  9.73it/s]


loss: 0.525552
acc: 0.744376
start validation


Loss: 0.42 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.44it/s]


validation loss: 0.5810795426368713
validation accuracy: 0.6984011627906976
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.486676
acc: 0.769049
start validation


Loss: 0.40 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.51it/s]


validation loss: 0.5899998545646667
validation accuracy: 0.6991279069767442
--------------------
* Epoch 11/100


Loss: 0.56 #nan->0:  32%|███▏      | 218/689 [00:24<00:52,  9.00it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi tanh dropout=0.7

* Epoch 1/100


Loss: 0.54 #nan->0: 100%|██████████| 689/689 [01:08<00:00, 10.00it/s]


loss: 0.694975
acc: 0.565312
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 12.18it/s]


validation loss: 0.6503216624259949
validation accuracy: 0.6002906976744186
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.300177
acc: 0.872823
start validation


Loss: 0.90 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.88it/s]


validation loss: 0.7846303582191467
validation accuracy: 0.7056686046511628
--------------------
* Epoch 7/100


Loss: 0.16 #nan->0:   6%|▌         | 38/689 [00:05<01:38,  6.60it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.86 #nan->0: 100%|██████████| 689/689 [01:59<00:00,  5.76it/s]


loss: 0.682115
acc: 0.560232
start validation


Loss: 0.77 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.64it/s]


validation loss: 0.6899991035461426
validation accuracy: 0.5472383720930233
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.179233
acc: 0.930697
start validation


Loss: 1.25 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.83it/s]


validation loss: 0.8176172971725464
validation accuracy: 0.7049418604651163
--------------------
* Epoch 9/100


Loss: 0.18 #nan->0:  62%|██████▏   | 430/689 [00:44<00:26,  9.67it/s]


KeyboardInterrupt: ignored

In [ ]:
#quello prima dalla 7°5° epoca (76%, 71%) ma con lr=1e-5
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-4_relu.bin'))
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.5 lr=1e-5

* Epoch 1/100


Loss: 0.70 #nan->0: 100%|██████████| 689/689 [01:10<00:00,  9.82it/s]


loss: 0.339140
acc: 0.855225
start validation


Loss: 0.55 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.72it/s]


validation loss: 0.68845134973526
validation accuracy: 0.717296511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.234502
acc: 0.905479
start validation


Loss: 0.82 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.56it/s]


validation loss: 0.8110164999961853
validation accuracy: 0.7151162790697675
--------------------
* Epoch 5/100


Loss: 0.37 #nan->0:  11%|█         | 73/689 [00:09<01:20,  7.63it/s]


KeyboardInterrupt: ignored

#### training con dataset sbagliato

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.55 #nan->0: 100%|██████████| 689/689 [02:01<00:00,  5.67it/s]


loss: 0.573213
acc: 0.724601
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 12.12it/s]


validation loss: 0.5274064540863037
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.232231
acc: 0.900943
start validation


Loss: 1.31 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.73it/s]


validation loss: 0.6630783677101135
validation accuracy: 0.7739825581395349
--------------------
* Epoch 10/100


  0%|          | 0/689 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.67 #nan->0: 100%|██████████| 689/689 [02:00<00:00,  5.73it/s]


loss: 0.691341
acc: 0.573113
start validation


Loss: 0.57 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.89it/s]


validation loss: 0.623408317565918
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.573078
acc: 0.732946
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.81it/s]


validation loss: 0.5683332681655884
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 10/100


Loss: 0.60 #nan->0:  91%|█████████ | 628/689 [01:03<00:06,  9.97it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu deep dropout=0.7 e 0.3

* Epoch 1/100


Loss: 0.38 #nan->0: 100%|██████████| 689/689 [02:02<00:00,  5.63it/s]


loss: 0.565083
acc: 0.730769
start validation


Loss: 0.45 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.78it/s]


validation loss: 0.5076175928115845
validation accuracy: 0.7558139534883721
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.047852
acc: 0.982221
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.63it/s]


validation loss: 1.0516650676727295
validation accuracy: 0.7819767441860465
--------------------
* Epoch 14/100


Loss: 0.07 #nan->0:   2%|▏         | 14/689 [00:03<02:40,  4.19it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 3secondi dataset relu

* Epoch 1/100


Loss: 1.08 #nan->0: 100%|██████████| 689/689 [01:58<00:00,  5.82it/s]


loss: 0.572104
acc: 0.720791
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.88it/s]


validation loss: 0.5387810468673706
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.019588
acc: 0.994376
start validation


Loss: 0.11 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.52it/s]


validation loss: 1.33577561378479
validation accuracy: 0.7914244186046512
--------------------
* Epoch 65/100


Loss: 0.08 #nan->0:   5%|▍         | 34/689 [00:05<01:45,  6.18it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 3secondi dataset balanced

* Epoch 1/100


Loss: 0.74 #nan->0: 100%|██████████| 387/387 [00:40<00:00,  9.65it/s]


loss: 0.710685
acc: 0.533592
start validation


Loss: 0.69 #nan->0: 100%|██████████| 97/97 [00:08<00:00, 11.15it/s]


validation loss: 0.662943959236145
validation accuracy: 0.6301546391752577
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.373187
acc: 0.836240
start validation


Loss: 0.14 #nan->0: 100%|██████████| 97/97 [00:08<00:00, 11.16it/s]


validation loss: 0.7514455914497375
validation accuracy: 0.720360824742268
--------------------
* Epoch 18/100


Loss: 0.42 #nan->0:   9%|▊         | 33/387 [00:05<00:55,  6.35it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.5 3secondi classificatore deep

* Epoch 1/100


Loss: 0.59 #nan->0: 100%|██████████| 689/689 [02:03<00:00,  5.57it/s]


loss: 0.546557
acc: 0.744557
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.50it/s]


validation loss: 0.5090526938438416
validation accuracy: 0.7638081395348837
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.054980
acc: 0.979318
start validation


Loss: 0.87 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.70it/s]


validation loss: 0.9800843596458435
validation accuracy: 0.7761627906976745
--------------------
* Epoch 11/100


Loss: 0.31 #nan->0:  25%|██▍       | 169/689 [00:19<00:58,  8.88it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.5 3secondi split=0.4

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 517/517 [01:40<00:00,  5.16it/s]


loss: 0.564747
acc: 0.732592
start validation


Loss: 0.32 #nan->0: 100%|██████████| 345/345 [00:23<00:00, 14.66it/s]


validation loss: 0.5135382413864136
validation accuracy: 0.7692028985507247
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.00 #nan->0: 100%|██████████| 517/517 [00:45<00:00, 11.35it/s]


loss: 0.021908
acc: 0.992747
start validation


Loss: 0.04 #nan->0: 100%|██████████| 345/345 [00:23<00:00, 14.62it/s]


validation loss: 1.2161269187927246
validation accuracy: 0.7927536231884058
--------------------
* Epoch 59/100


Loss: 0.00 #nan->0:  77%|███████▋  | 396/517 [00:35<00:10, 11.86it/s]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 6secondi

* Epoch 1/100


Loss: 0.87 #nan->0: 100%|██████████| 689/689 [01:16<00:00,  9.04it/s]


loss: 0.593195
acc: 0.711901
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.91it/s]


validation loss: 0.5598057508468628
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.291185
acc: 0.876996
start validation


Loss: 0.82 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 14.26it/s]


validation loss: 0.5884414911270142
validation accuracy: 0.7659883720930233
--------------------
* Epoch 16/100


Loss: 0.08 #nan->0:  32%|███▏      | 220/689 [00:25<00:54,  8.63it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 6secondi

* Epoch 1/100


Loss: 0.47 #nan->0: 100%|██████████| 689/689 [01:15<00:00,  9.11it/s]


loss: 0.560946
acc: 0.730588
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.68it/s]


validation loss: 0.5140737295150757
validation accuracy: 0.7521802325581395
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.210140
acc: 0.914731
start validation


Loss: 0.57 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.61it/s]


validation loss: 0.6174583435058594
validation accuracy: 0.7659883720930233
--------------------
* Epoch 7/100


Loss: 0.06 #nan->0:  14%|█▍        | 98/689 [00:12<01:13,  8.04it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.5

* Epoch 1/100


Loss: 0.58 #nan->0: 100%|██████████| 689/689 [02:03<00:00,  5.60it/s]


loss: 0.554238
acc: 0.731858
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.69it/s]


validation loss: 0.5173416137695312
validation accuracy: 0.7390988372093024
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.022887
acc: 0.990747
start validation


Loss: 1.72 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.10it/s]


validation loss: 1.19481360912323
validation accuracy: 0.7725290697674418
--------------------
* Epoch 35/100


Loss: 0.00 #nan->0:  62%|██████▏   | 427/689 [00:46<00:28,  9.09it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz EXTENDED dropout=0.5

* Epoch 1/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [02:30<00:00,  4.57it/s]


loss: 0.549004
acc: 0.742017
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.59it/s]


validation loss: 0.5087576508522034
validation accuracy: 0.7521802325581395
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.191890
acc: 0.921626
start validation


Loss: 0.56 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.60it/s]


validation loss: 0.6840699911117554
validation accuracy: 0.7805232558139535
--------------------
* Epoch 6/100


Loss: 0.55 #nan->0:  15%|█▌        | 104/689 [00:23<02:13,  4.38it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz EXTENDED

* Epoch 1/100


Loss: 0.46 #nan->0: 100%|██████████| 689/689 [03:24<00:00,  3.37it/s]


loss: 0.570584
acc: 0.725871
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.61it/s]


validation loss: 0.5190539956092834
validation accuracy: 0.7492732558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.046427
acc: 0.984216
start validation


Loss: 1.44 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.58it/s]


validation loss: 0.9704733490943909
validation accuracy: 0.7936046511627907
--------------------
* Epoch 12/100


Loss: 0.15 #nan->0:  98%|█████████▊| 677/689 [02:28<00:02,  4.56it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz

* Epoch 1/100


Loss: 0.62 #nan->0: 100%|██████████| 689/689 [01:11<00:00,  9.63it/s]


loss: 0.573839
acc: 0.726415
start validation


Loss: 0.48 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.44it/s]


validation loss: 0.5224190354347229
validation accuracy: 0.7507267441860465
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.064792
acc: 0.976597
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.24it/s]


validation loss: 1.109357476234436
validation accuracy: 0.7638081395348837
--------------------
* Epoch 12/100


Loss: 0.01 #nan->0:  91%|█████████ | 626/689 [01:07<00:06,  9.26it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#7mhz

* Epoch 1/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [01:26<00:00,  7.93it/s]


loss: 0.559725
acc: 0.724964
start validation


Loss: 0.33 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 15.22it/s]


validation loss: 0.497946172952652
validation accuracy: 0.7623546511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.109533
acc: 0.960450
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 15.29it/s]


validation loss: 0.7650411128997803
validation accuracy: 0.7936046511627907
--------------------
* Epoch 9/100


Loss: 0.01 #nan->0:   4%|▍         | 28/689 [00:05<01:59,  5.55it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#5mhz

* Epoch 1/100


Loss: 0.76 #nan->0: 100%|██████████| 689/689 [01:22<00:00,  8.38it/s]


loss: 0.572365
acc: 0.724057
start validation


Loss: 0.52 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.20it/s]


validation loss: 0.5210419297218323
validation accuracy: 0.7420058139534884
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.099157
acc: 0.962083
start validation


Loss: 1.81 #nan->0: 100%|██████████| 172/172 [00:17<00:00, 10.06it/s]


validation loss: 0.7372713088989258
validation accuracy: 0.8081395348837209
--------------------
* Epoch 10/100


Loss: 0.04 #nan->0:  11%|█         | 74/689 [00:10<01:31,  6.74it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#2mhz

* Epoch 1/100


Loss: 0.44 #nan->0: 100%|██████████| 689/689 [00:55<00:00, 12.52it/s]


loss: 0.577725
acc: 0.726778
start validation


Loss: 0.52 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 15.24it/s]


validation loss: 0.5585514307022095
validation accuracy: 0.7325581395348837
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.050927
acc: 0.980951
start validation


Loss: 0.94 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 14.95it/s]


validation loss: 1.3156260251998901
validation accuracy: 0.7463662790697675
--------------------
* Epoch 16/100


Loss: 0.01 #nan->0:   5%|▌         | 35/689 [00:04<01:27,  7.51it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#3mhz

* Epoch 1/100


Loss: 0.85 #nan->0: 100%|██████████| 689/689 [01:45<00:00,  6.53it/s]


loss: 0.588611
acc: 0.723331
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.41it/s]


validation loss: 0.5512516498565674
validation accuracy: 0.7238372093023255
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.180153
acc: 0.932511
start validation


Loss: 1.28 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 18.14it/s]


validation loss: 0.8466366529464722
validation accuracy: 0.7572674418604651
--------------------
* Epoch 8/100


Loss: 0.04 #nan->0:  27%|██▋       | 188/689 [00:15<00:42, 11.92it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])

* Epoch 1/100


Loss: 0.60 #nan->0: 100%|██████████| 689/689 [00:53<00:00, 12.91it/s]


loss: 0.604355
acc: 0.714623
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.87it/s]


validation loss: 0.572152853012085
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.088003
acc: 0.968070
start validation


Loss: 1.85 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.21it/s]


validation loss: 1.095919132232666
validation accuracy: 0.7616279069767442
--------------------
* Epoch 37/100


Loss: 0.00 #nan->0:  84%|████████▍ | 581/689 [00:46<00:08, 12.61it/s]


KeyboardInterrupt: ignored